In [ ]:
import os


GOOGLE_API_KEY = "API_KEY"
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("✅ Setup and authentication complete.")

✅ Setup and authentication complete.


In [2]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.tools import AgentTool, FunctionTool, google_search,load_memory
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.genai import types


print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [5]:
memory_service = (
    InMemoryMemoryService()
)

In [6]:
APP_NAME = "MemoryResearcher"
USER_ID = "user-1"

In [7]:
initial_researcher_agent = Agent(
    name="ResearcherAgent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction="""Act as a leading theoretical researcher. Based on the provided topic, formulate a novel theorem that addresses a gap in current understanding.
        Structure your response in two distinct sections:
        The Theorem: A concise statement of the new idea (approx. 100 words).
        The Derivation/Solution: A step-by-step logical or mathematical framework that substantiates how this theorem functions or how it was derived. 
        Use LaTeX for any necessary formalism.

        Constraints:
        Output only the content of these two sections. No conversational filler.""",\
    output_key="current_research",  
)

print("✅ research_agent created.")

✅ research_agent created.


In [8]:
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction="""You are a research critic. Review the research provided below.
    Research: {current_research}
    
    Evaluate the research's novelty, idea.
    - Ensure the idea is novel use google_search tool to check this; if a similar concept exists, extend it to a specific, unexplored edge case.
    - If the research is well-written  and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    tools=[google_search],
    output_key="critique",  
)

print("✅ critic_agent created.")

✅ critic_agent created.


In [9]:
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the research is finished and no more changes are needed."""
    return {"status": "approved", "message": "Research approved. Exiting refinement loop."}


print("✅ exit_loop function created.")

✅ exit_loop function created.


In [10]:
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction="""You are a research refiner. You have a research draft and critique.
    
    Research Draft: {current_research}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the research draft to fully incorporate the feedback from the critique.""",
    output_key="current_research",  
    tools=[
        FunctionTool(exit_loop)
    ], 
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


In [11]:
research_refinement_loop = LoopAgent(
    name="ResearchRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  
)


In [12]:
writer_agent_1 = Agent(
    name="AbstractWriter",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction=""" You are abstract writer.
    Research Draft: {current_research}
    What novelty does this research have? Using them make Abstract for paper (around 50-100 words).
    Constraints:
    Output only the content. No conversational filler
    """,
    output_key="abstract_for_paper",
)

writer_agent_2 = Agent(
    name="ResearchWriter",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction=""" You are research writer.
    Research Draft: {current_research}
    With what research this draft came up, use them and write paper in scientific manner, with calculations and etc. (around 150-200 words).
    Constraints:
    Output only the content. No conversational filler
    """,
    output_key="body_for_paper",
)

writer_agent_3 = Agent(
    name="ReferenceWriter",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction=""" You are reference writer.
    Research Draft: {current_research}
    Analyzing this what idea used in this text what references possible used, find them using google_search tool 
    and in numerically order write reference part for the paper.
    
    Constraints:
    Output only the content. No conversational filler
    """,
    tools=[google_search],
    output_key="reference_for_paper",
)

print("✅ Writer agents created")

✅ Writer agents created


In [13]:
parallel_writer_agent = ParallelAgent(
     name="ParallelWriterAgent",
     sub_agents=[writer_agent_1, writer_agent_2, writer_agent_3],
     description="Runs multiple writer agents in parallel to write complete paper."
 )

print("✅ Parallel writer agent created")

✅ Parallel writer agent created


In [14]:
main_writer_agent = Agent(
    name="MainWriter",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction=""" You are latex writer agent. Write latex paper from provided texts 
    Abstract part {abstract_for_paper}
    Body part {body_for_paper}
    Reference part {reference_for_paper}
    
    With all of this write ready paper in latex format.
    .
    Constraints:
    Output only the content. No conversational filler
    """,
)

In [15]:
root_agent = SequentialAgent(
    name="ResearchPipeline",
    sub_agents=[initial_researcher_agent, research_refinement_loop, parallel_writer_agent, main_writer_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [16]:
session_service = InMemorySessionService()  # Handles conversations

# Create runner with BOTH services
runner = Runner(
    agent=root_agent,
    app_name="MemoryResearcher",
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
)

print("✅ Agent and Runner created with memory support!")

✅ Agent and Runner created with memory support!


In [17]:
await run_session(
    runner,
    "Write a research related to mathematics, especially related to derivatives",
    "research-01",  # Session ID
)


### Session: conversation-01

User > Write a research related to mathematics, especially related to derivatives
Model: > ### The Theorem

**The Theorem of Fractional Derivative Proximity.** For a function $f(t) \in C^n[a, b]$, where $n \in \mathbb{Z}^+$, the Caputo fractional derivative of order $\alpha$, denoted ${^C}D^\alpha_a f(t)$, converges to the standard integer-order derivative $f^{(n-1)}(t)$ as $\alpha \to (n-1)^+$. This convergence is not abrupt; the deviation is bounded and linearly proportional to the fractional excess $\epsilon = \alpha - (n-1)$. The theorem states that the normalized difference is bounded as:
$|{^C}D^\alpha_a f(t) - (f^{(n-1)}(t) - f^{(n-1)}(a))| \le M_n (\alpha - (n-1)) \mathcal{C}(t,a) + \mathcal{O}((\alpha - (n-1))^2)$,
where $M_n = \sup_{\tau \in [a,b]} |f^{(n)}(\tau)|$ and $\mathcal{C}(t,a)$ is a computable constant dependent on the interval.

### The Derivation/Solution

The substantiation of the theorem proceeds by analyzing the definitions of the

Model: > 

Model: > This paper presents a direct and rigorous derivation for the linear rate of convergence of the Caputo fractional derivative to its integer-order counterpart as the fractional order approaches an integer. Our contribution is a formal proof methodology using Taylor's theorem with explicit remainders to establish an error bound linearly proportional to the fractional excess. A novel analysis of the resulting bounding constant, including its explicit analytical form, is also provided. This work offers a robust and pedagogical demonstration of a known property, valuable for applications where approximation error is a key concern.
Model: > A foundational property of fractional calculus is the convergence of fractional derivatives to their integer-order counterparts as the order aligns. This paper presents a direct and rigorous derivation of the linear convergence rate for the Caputo fractional derivative, ${^C}D^\alpha_a f(t)$, as its order $\alpha \to (n-1)^+$. We analyz